<a href="https://colab.research.google.com/github/ManuelBagasina/solarspell/blob/playground/RwandaB2RClean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is how we are translating our document to English from Kinyarwandan. The first step will be to upload the file needed to the google colab machine, as the SolarSPELL Library does not allow for connecting to the drive directly to source files.

In [ ]:
from google.colab import files
uploaded = files.upload()

TypeError: 'NoneType' object is not subscriptable

This next step is naming the CSV you uploaded with the ***exact name of the file you uploaded.*** This whole process will not work if you give a differernt name.

In [ ]:
import pandas as pd

# Replace with the exact name of the file you uploaded
df = pd.read_csv('Copy of SolarSPELL copy_ B2R Second Farmer Field Visits Responses - Form Responses 1.csv')


This installs the translator tool we will be using for the remainder of this cleaning.

In [ ]:
!pip install deep-translator



Next, will be running our code to translate this data. This is not fast, considering this file in specific will be translating G2:AW3727. Find a good book or something, because this will take ___ minutes roughly, on a Fiber WiFi connection.

In [ ]:
from deep_translator import GoogleTranslator
import pandas as pd

def safe_translate(text, dest='en'):
    if pd.isna(text) or str(text).strip() == '':
        return text
    try:
        return GoogleTranslator(source='auto', target=dest).translate(str(text))
    except Exception as e:
        print(f"Translation error for '{text}': {e}")
        return text

def translate_columns(df, cols):
    for col in cols:
        print(f"Translating column: {col}")
        df[col] = df[col].apply(safe_translate)
    return df

df = pd.read_csv('Copy of SolarSPELL copy_ B2R Second Farmer Field Visits Responses - Form Responses 1.csv')

start_idx = 6
end_idx = 49
cols_to_translate = df.columns[start_idx:end_idx]

df = translate_columns(df, cols_to_translate)

df.to_csv('SolarSPELL_Translated_FullEnglish.csv', index=False)
